In [2]:
! ipython kernel install --user --name=translational_variability_3.9

Installed kernelspec translational_variability_3.9 in /home/warri/.local/share/jupyter/kernels/translational_variability_3.9


In [9]:
! which python | echo > ./output.txt

In [1]:
from easynmt import EasyNMT
model = EasyNMT('m2m_100_418M')

In [6]:
sentence = "Он положил свою руку к ней на плечо"
model.translate(sentence, source_lang='ru', target_lang='en', beam_size=15, max_length=200)

'He put his hand on her shoulder.'

In [1]:
from simalign import SentenceAligner
myaligner = SentenceAligner(model="bert", token_type="bpe", matching_methods="mai")

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2023-04-16 01:00:26,933 - simalign.simalign - INFO - Initialized the EmbeddingLoader with model: bert-base-multilingual-cased


In [2]:
with open("./corpora/karenina/originals/Books_ru.txt", "rt", encoding="utf-8") as file:
    ru_corpus_raw = [line.rstrip() for line in file]

In [3]:
with open("./corpora/karenina/ru_tokens.txt", "rt", encoding="utf-8") as file:
    ru_tokens = [line.rstrip() for line in file]

In [8]:
! python3 -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 966.6 kB/s eta 0:00:00m eta 0:00:010:00:07
You should consider upgrading via the '/home/warri/code/aaa/translational_variability/translational_variability_3.9/bin/python3 -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [4]:
from tqdm import tqdm
import spacy

In [5]:
en_nlp_lg = spacy.load("en_core_web_lg")

In [14]:
translation_text = ""
docs = []

for i in tqdm(range(8522, len(ru_corpus_raw))):
    translation = model.translate(ru_corpus_raw[i], source_lang='ru', target_lang='en', beam_size=10, max_length=200)
    spacy_doc = en_nlp_lg(translation)
    docs.append(spacy_doc)
    translation_text += translation
    translation_text += '\n'

100%|██████████| 8733/8733 [18:06:28<00:00,  7.46s/it]   


In [15]:
with open('./translations_karenina_facebook_beam10.txt', 'wt', encoding="utf-8") as f:
    f.write(translation_text)

from spacy.tokens import DocBin
doc_bin = DocBin()
for doc in docs:
    doc_bin.add(doc)
bytes_data = doc_bin.to_bytes()

with open("./translated_spacy_doc_beam10_facebook.bin", "wb") as file:
    file.write(bytes_data)

In [13]:
translation_facebook = translation_text
doc_facebook = docs

In [9]:
with open('./translations_karenina_facebook_beam15.txt', 'wt', encoding="utf-8") as f:
    f.write(translation_text)

from spacy.tokens import DocBin
doc_bin = DocBin()
for doc in docs:
    doc_bin.add(doc)
bytes_data = doc_bin.to_bytes()

with open("./translated_spacy_doc_beam15_facebook.bin", "wb") as file:
    file.write(bytes_data)

In [1]:
mwmf = ""
itermax = ""
inter = ""

for i, doc in tqdm(enumerate(docs)):
    en_tokens = [word.text for word in doc]
    print(ru_tokens[i], "\n", en_tokens)
    src, trg = (ru_tokens[i], en_tokens)
    alignments = myaligner.get_word_aligns(src, trg)
    mwmf += " ".join([f"{x}-{y}" for x, y in alignments["mwmf"]])
    itermax += " ".join([f"{x}-{y}" for x, y in alignments["itermax"]])
    inter += " ".join([f"{x}-{y}" for x, y in alignments["inter"]])
    mwmf += "\n"
    itermax += "\n"
    inter += "\n"

NameError: name 'tqdm' is not defined

In [17]:
with open('./mwmf.txt', 'wt') as f:
    f.write(mwmf)
with open('./itermax.txt', 'wt') as f:
    f.write(itermax)
with open('./inter.txt', 'wt') as f:
    f.write(inter)

In [14]:
with open('./translations_karenina_facebook_beam15.txt', 'wt') as f:
    f.write(translation_text)

In [ ]:
translation_list2 = []
for line in tqdm(ru_corpus_raw[5000:10000]):
    translation = model.translate(line, source_lang="ru", target_lang='en', beam_size=20)
    translation_list2.append(translation)

In [ ]:
translation_list3 = []
for line in tqdm(ru_corpus_raw[10000:]):
    translation = model.translate(line, source_lang="ru", target_lang='en', beam_size=20)
    translation_list3.append(translation)

In [18]:
from spacy.tokens import DocBin

doc_bin = DocBin()
for doc in docs:
    doc_bin.add(doc)
bytes_data = doc_bin.to_bytes()

with open("./translated_spacy_doc_beam15.bin", "wb") as file:
    file.write(bytes_data)

In [ ]:
translation_data = ["" for i in range(len(ru_corpus_raw))]
aligned_data_itermax = ["" for i in range(len(ru_corpus_raw))]
aligned_data_inter = ["" for i in range(len(ru_corpus_raw))]
aligned_data_mwmf = ["" for i in range(len(ru_corpus_raw))]
for idx, translation, alignments in data[2]:
    translation_data[idx] = translation
    aligned_data_mwmf[idx] = " ".join([f"{x}-{y}" for x, y in alignments["mwmf"]])
    aligned_data_itermax[idx] = " ".join([f"{x}-{y}" for x, y in alignments["itermax"]])
    aligned_data_inter[idx] = " ".join([f"{x}-{y}" for x, y in alignments["inter"]])